In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


아니그니까, 잘 가다가 이상한데로 간(다른 차원?) 사람들을 찾으라는건데
저온수면중이면 갇혀있었고 = 다른데로 못감
VIP는 VIP대우를 받아서 잘 옮겨졌고(?)
그 나머지를 찾는건데... 
- 저온수면 + vip 모두 잘 옮겨졌으니 빼도 되나?
- 쓸데없는 cabin 이런거 값 필요없어 ?
- 결측치가 반 이상이면 솔직히 필요없어 보여 ?
- homeplanet, destination 연관성 ?


# Step1.라이브러리 및 데이터 불러오기
- 라이브러리 버전 확인!

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')# 경고메세지 필요없음

print(pd.__version__)
print(np.__version__)

1.3.5
1.20.3


In [3]:
import pandas as pd
submission = pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [4]:
train= pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train.head()
# 모델 학습 에 필요한 데이터 
# Target = Transported

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [5]:
test= pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
test.head()
# 모델 적용 대상 데이터

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [8]:
print(f'\033[94m train set have {train.shape[0]} rows and {train.shape[1]} columns.') #문자색 지정
print(f'\033[94m test set have {test.shape[0]} rows and {test.shape[1]} columns.') 
print(f'\033[94m submission set have {submission.shape[0]} rows and {submission.shape[1]} columns.') 

 train set have 8693 rows and 14 columns.
 test set have 4277 rows and 13 columns.
 submission set have 4277 rows and 2 columns.


In [9]:
print(f'\033[31m') # 문자색 지정
print((train.isna().sum().sort_values(ascending = False))) # 내림차순 정렬, 결측치 합
print(f'\033[32m')
print((test.isna().sum().sort_values(ascending = False)))


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64

FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


In [10]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [11]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


# Step2. 탐색적 자료 분석(EDA)
- 데이터 시각화
- 산점도, 막대그래프, 등등
- 그래프 해석해서 설명을 달아야 함
- 약간의 데이터 전처리

- 수치형

In [12]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=train.select_dtypes(include=numeric)
df_num.columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')

In [13]:
numeric=['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_num=test.select_dtypes(include=numeric)
df_num.columns

Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')

- 범주형

In [14]:
df_cat = train.select_dtypes(include = 'object')
df_cat.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,False,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,False,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,True,Altark Susent


In [15]:
df_cat = test.select_dtypes(include = 'object')
df_cat.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,False,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,False,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,False,Sabih Unhearfus


- 지울거, 범주형 필요없음..일단 그래보임

- missing value 찾아야하고..

# Step3. 데이터 전처리
- Feature Engineering
- 머신러닝 모형을 돌리기 위해서 표준화, 원핫-인코딩 등
- 파생변수 (도출변수) 만들기
    + 왜 이 변수를 만들었는지에 대한 나의 설명이 필요

# Step4. 머신러닝 모형 개발
- 모형쓸거아냐 모형에 대한 설명이 들어가 있어야 함
- 모형 1-2개 정도만 써라
- 교차검증, 하이퍼파라미터튜닝, 등등 훈련데이터 등등

# Step5. 모형 평가
- 훈련데이터를 쪼갠다. 훈련데이터 + 검증데이터 가지고 평가
- 테스트 데이터 건드는거 아님
- 정확도 비교
- 혼동행렬 (Confusion Martix) 설명


# Step6. 제출
- 제출 양식은 만들어주신대(샘플)

# 참고
- 다른 사람의 코드 설명을 보고 쭉 따라쳤어 만약에
- 코드 참조 했다고 명시해야함 노트북 표절 방지
- 코드 링크 걸기
- 저자이름, 글제목, 링크주소 ...

# 마감일
- 04.12 17:40 까지
- 제출 형태
    + 개인 DM으로 등수 그거 캡쳐 리더보드
    + 고용노동부 보고 양식 (다음주에 확인해서 알려주신대)